In [ ]:
# instructions to download miniconda3 per their website

# from the terminal run these lines of code:
"""
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm ~/miniconda3/miniconda.sh

source ~/miniconda3/bin/activate
conda init --all
"""

# then close and reopen the terminal

In [ ]:
# set up MetaXcan

# clone the repository
"""
cd ~
git clone https://github.com/hakyimlab/MetaXcan.git
"""

# create a conda environment for MetaXcan
"""
cd MetaXcan/software
conda env create -f conda_env.yaml
"""

# then, open up a jupyter notebook
# click on the 'Kernel' tab at the top
# hover over 'Change kernel'
# click on 'Python [conda env:imlabtools]'

In [ ]:
# get covariance matrices via

"""
wget https://zenodo.org/record/3518299/files/mashr_eqtl.tar?download=1 -O mashr_eqtl.tar
tar -xvpf mashr_eqtl.tar
"""

# transcriptome prediction model database and snp covariance matrices contained therein

In [ ]:
# format data for MetaXcan

import os
print(os.getcwd())
os.chdir("/home/jupyter/packages/MetaXcan/software/261-run")
print(os.getcwd())
os.system("ls")

import pandas as pd
df = pd.read_csv("test.tsv", sep="\t")
eff = []
neff = []
for row in df["alleles"]:
    eff.append(row[2])
    neff.append(row[6])

print(eff)
print(neff)

df["effect"] = eff
df["non-effect"] = neff
df.drop("alleles", axis=1, inplace=True)

print(df)

In [ ]:
# run S-PrediXcan with test data
import os
os.chdir("/home/jupyter/MetaXcan/software")

os.system("./SPrediXcan.py \
--model_db_path 261-run/mashr_Stomach.db \
--covariance 261-run/mashr_Stomach.txt.gz \
--gwas_folder 261-run/filtered_261.2.tsv \
--snp_column locus \
--effect_allele_column effect \
--non_effect_allele_column non-effect \
--beta_column BETA \
--pvalue_column Pvalue \
--output_file results/test.csv")